# Api Twitter
* https://datascienceparichay.com/article/get-data-from-twitter-api-in-python-step-by-step-guide/
* https://developer.twitter.com/en/portal/projects/1391772372416532480/apps/20820453/keys

Análisis de sentimiento en Español:
* https://platzi.com/tutoriales/1874-python-lenguaje-natural/5654-realiza-un-analisis-de-sentimiento-en-3-pasos-con-python/
    * Ejemplo https://twitter.com/whaleandjaguar_?lang=en

In [2]:
import tweepy as tw
import getpass

In [3]:
my_api_key=getpass.getpass()

 ·························


In [4]:
my_api_secret=getpass.getpass()

 ··················································


In [5]:
auth = tw.OAuthHandler(my_api_key, my_api_secret)

In [6]:
api = tw.API(auth, wait_on_rate_limit=True)

In [28]:
search_query = "#SOSColomia OR #ParoNacional OR #Cali"

In [31]:
# get tweets from the API
tweets = tw.Cursor(api.search,
              q=search_query,
              lang="es",
              since="2021-03-01" #https://es.wikipedia.org/wiki/Protestas_en_Colombia_de_2021
                  ).items(500) 

In [32]:
# store the API responses in a list
tweets_copy = []
for tweet in tweets:
    tweets_copy.append(tweet)
    
print("Total Tweets fetched:", len(tweets_copy))

Total Tweets fetched: 500


In [27]:
import pandas as pd

# intialize the dataframe
tweets_df = pd.DataFrame()

# populate the dataframe
for tweet in tweets_copy:
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source}))
    tweets_df = tweets_df.reset_index(drop=True)

# show the dataframe
tweets_df.head()

,user_name,user_location,user_description,user_verified,date,text,hashtags,source
0,Maria Paula Quiceno,Bogota,Pienso en innovación para los socioecosistemas...,False,2021-05-10 21:00:06,Dolor! @CacaoHatun\n#comisioninteramericanaded...,"[comisioninteramericanadederechoshumanos, Euro...",Twitter for Android
1,Nancy Ruiz-Uribe,"Ithaca, NY",PhD candidate at @CornellBME studying Alzheime...,False,2021-05-10 20:50:58,If you would like to help #Colombians in need ...,"[Colombians, ParoNacional]",Twitter Web App
2,AgnosioFronemos,,"Estimado mamerto me importa un qlo, si le alca...",False,2021-05-10 20:24:55,@BLUPacifico @IvanDuque On your face #ParoNaci...,[ParoNacional],Twitter Web App
3,THE CITY PAPER,"Bogotá, Colombia",Colombia's longest running English language ne...,False,2021-05-10 20:10:18,President Iván Duque calls on Mayors to left b...,None,Twitter Web App
4,Jairo Eduardo Castro,Mocoa-Putumayo-Colombia,"Director de Proyectos TICs, Gerente Inmobiliar...",False,2021-05-10 20:08:52,Razonable #ParoNacional10M #ParoNacional #Paro...,"[ParoNacional10M, ParoNacional, ParoMataBebes,...",Twitter for Android


In [18]:
import pandas as pd

# intialize the dataframe
tweets_df = pd.DataFrame()

# populate the dataframe
for tweet in tweets_copy:
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source}))
    tweets_df = tweets_df.reset_index(drop=True)

# show the dataframe
tweets_df.head()

'Twitter for Android'

In [23]:
tweet.favorite.

<bound method Status.favorite of Status(_api=<tweepy.api.API object at 0x7f68125b18d0>, _json={'created_at': 'Mon May 10 16:06:22 +0000 2021', 'id': 1391786701513576453, 'id_str': '1391786701513576453', 'text': 'In 365 days, India’s #COVID19 case tally has inflated by a factor of 514, up from 37,257 to 19,164,969, and a total… https://t.co/wTnOqsu3tn', 'truncated': True, 'entities': {'hashtags': [{'text': 'COVID19', 'indices': [21, 29]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/wTnOqsu3tn', 'expanded_url': 'https://twitter.com/i/web/status/1391786701513576453', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': 